# Convertible Bond Valuation - QL Example

Value a Convertible Bond and compare to QL example at http://gouthamanbalaraman.com/blog/value-convertible-bond-quantlib-python.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.bonds import *
from financepy.market.curves import *

## Creating the Convertible Bond

In [3]:
value_dt = Date(9, 1, 2004)
settle_dt = value_dt.add_days(2)

We set the maturity date, annualised coupon, coupon annual_frequency and convention for calculating accrued interest

In [4]:
maturity_dt = Date(15, 3, 2022)
coupon = 0.0575
freq_type = FrequencyTypes.SEMI_ANNUAL
accrualBasis = DayCountTypes.THIRTY_360_BOND
face = 100.0
conversion_ratio = 3.84615

We need to set the face amount and the conversion ratio. This is the number of shares you can convert to for that face amount.

We can also delay the start of the conversion feature using the start convert date. We allow conversion to occur from settlement.

In [5]:
start_convert_dt = settle_dt

We can also set the call schedule - the price and date on which the issuer can call back the bond. This is given with reference to a price of 100. Making the call price very high means it will never be called. So the call has no impact.

In [6]:
call_price = 100.0
call_dts = [Date(20, 3, 2007), Date(15, 3, 2012), Date(15, 3, 2017)]
call_prices = np.array([call_price, call_price, call_price])

We can also set the put schedule - the price and date on which the owner can put back the bond to the issuer. This is given with reference to a price of 100. Making the put price very low means it will never be put.

In [7]:
putPrice = 100.0
put_dts = [Date(20, 3, 2007), Date(15, 3, 2012), Date(15, 3, 2017)]
put_prices = np.array([putPrice, putPrice, putPrice])

At this point we can create our bond.

In [8]:
bond = BondConvertible(maturity_dt, coupon, freq_type, start_convert_dt, conversion_ratio, call_dts, call_prices, put_dts, put_prices, accrualBasis)

In [9]:
print(bond)

OBJECT TYPE: BondConvertible
MATURITY DATE: 15-MAR-2022
COUPON: 0.0575
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT TYPE: DayCountTypes.THIRTY_360_BOND
CONVERSION RATIO: 3.84615
START CONVERT DATE: 11-JAN-2004
CALL: DATES
20-MAR-2007: 100.0
15-MAR-2012: 100.0
15-MAR-2017: 100.0
PUT: DATES
20-MAR-2007: 100.0
15-MAR-2012: 100.0
15-MAR-2017: 100.0



## Valuation Inputs

The model allows a discrete dividend schedule. We can use the FinSchedule class to do this.

In [10]:
dividend_schedule = Schedule(settle_dt, maturity_dt).schedule_dts()[1:]
dividend_yields = np.array([0.02] * len(dividend_schedule))

In [11]:
rate = 0.04
discount_curve = DiscountCurveFlat(settle_dt, rate, FrequencyTypes.CONTINUOUS)

In [12]:
stock_price = 29.04
stock_volatility = 0.40
credit_spread = 0.03
recovery_rate = 0.40 # 40 percent
num_steps_per_year = 1000

In [13]:
bond.value(value_dt, stock_price, stock_volatility, dividend_schedule, dividend_yields,
           discount_curve, credit_spread, recovery_rate, num_steps_per_year)

{'cbprice': np.float64(135.35452399080853),
 'bond': np.float64(89.44004150041206),
 'delta': np.float64(2.8964034274730124),
 'gamma': np.float64(0.5948917917094881),
 'theta': np.float64(221.62044995424202)}

Copyright (c) 2020 Dominic O'Kane

This compares to 132.31 found by QL. The difference could be due to the different treatment of dividends or the credit spread.